# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#bring in the csv that I created from the first part of the project and create a new dataframe

weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Trairi,-3.28,-39.27,86.85,48,0,13.11,BR,1595003273
1,Saskylakh,71.92,114.08,49.98,87,100,10.80,RU,1595003496
2,Hilo,19.73,-155.09,73.40,83,90,4.70,US,1595003270
3,Iqaluit,63.75,-68.51,50.00,100,90,2.24,CA,1595003497
4,Te Anau,-45.42,167.72,30.56,94,0,0.85,NZ,1595003497


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#set everything up to use gmaps
gmaps.configure(api_key = g_key)

locations = weather_df[['Lat', 'Lng']].astype(float)

humidity = weather_df['Humidity'].astype(float)


In [4]:
#add the heat layer on gmaps
fig = gmaps.figure(center = (0,0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create a filter to find the ideal locations
ideal_weather_df = weather_df.loc[(weather_df['Max Temp']>=75) & (weather_df['Max Temp']<=80) &\
                                  (weather_df['Wind Speed']<=10) & (weather_df['Cloudiness']<=40) &\
                                  (weather_df['Cloudiness']>=20) & (weather_df['Humidity']<=80) &\
                                  (weather_df['Humidity']>=50)].dropna()

ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,Margate,51.38,1.39,79.00,56,40,3.00,GB,1595003288
67,Cambridge,43.36,-80.31,79.00,61,20,6.93,CA,1595003240
72,Ponta do Sol,32.67,-17.10,75.99,69,20,5.82,PT,1595003512
75,San Juan del Paraná,-27.30,-55.97,78.80,65,20,5.82,PY,1595003512
154,São João da Barra,-21.64,-41.05,78.80,50,20,4.70,BR,1595003532
158,Chamba,32.57,76.13,76.12,63,28,2.93,IN,1595003533
170,Sant Feliu de Guíxols,41.78,3.03,75.99,54,21,8.63,ES,1595003258
210,Kahului,20.89,-156.47,75.00,73,20,6.93,US,1595003546
271,Waterloo,43.47,-80.52,79.00,61,20,6.93,CA,1595003449
325,Palora,-1.70,-77.93,77.00,61,20,3.36,EC,1595003573


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create a new column that we will fill with hotel names in each particular area
hotel_df = ideal_weather_df.loc[:,['City','Country','Lat','Lng']]

hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
34,Margate,GB,51.38,1.39,
67,Cambridge,CA,43.36,-80.31,
72,Ponta do Sol,PT,32.67,-17.10,
75,San Juan del Paraná,PY,-27.30,-55.97,
154,São João da Barra,BR,-21.64,-41.05,
158,Chamba,IN,32.57,76.13,
170,Sant Feliu de Guíxols,ES,41.78,3.03,
210,Kahului,US,20.89,-156.47,
271,Waterloo,CA,43.47,-80.52,
325,Palora,EC,-1.70,-77.93,


In [7]:
#set up parameters for my search
params = {'key': g_key,
          'radius': 5000,
          'type': 'hotel',
          'keyword': 'hotel'
         }

#use iterrows to go through each row of the dataframe and find the closest hotel
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = F"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params = params).json()
    
    #fill in the empty column with the data that we are able to find
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
34,Margate,GB,51.38,1.39,Sands Hotel Margate
67,Cambridge,CA,43.36,-80.31,Langdon Hall Country House Hotel & Spa
72,Ponta do Sol,PT,32.67,-17.10,Estalagem Da Ponta Do Sol
75,San Juan del Paraná,PY,-27.30,-55.97,Del Sur Hotel - Museo
154,São João da Barra,BR,-21.64,-41.05,Pousada Kactus
158,Chamba,IN,32.57,76.13,Hotel City Heart
170,Sant Feliu de Guíxols,ES,41.78,3.03,Hotel Planamar
210,Kahului,US,20.89,-156.47,Maui Beach Hotel
271,Waterloo,CA,43.47,-80.52,Courtyard by Marriott Waterloo St. Jacobs
325,Palora,EC,-1.70,-77.93,Finca La Esperanza


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure

fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))